# script to write normalized counts from diffbind dataset

In [2]:
library("tidyverse")
library("DiffBind")
library("readxl")
library("pheatmap")
options(bitmapType="cairo")

setwd("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/ATAC_seq_analysis")


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.1.0
✔ tidyr   1.1.4     ✔ stringr 1.4.1
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, pa

In [7]:
dataset_info_file = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/metadata/cleaned_ATAC_metadata.csv"
dataset_info = read.csv(dataset_info_file)
dataset_peaks_location = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/master_ATAC_ChIP_analyzer/macs2"
dataset_alignment_location = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/master_ATAC_ChIP_analyzer/clean_alignments"

dataset_info <- dataset_info %>% filter(condition %in% c("healthy", "patient", "synovium"))

data_for_diffbind <- dataset_info %>% select(id, patient, cell_type, condition, active_disease, female_sex, on_bDMARD_tsDMARD,
on_csDMARD, on_steroid, On_MTX, group, age, disease_duration, cell_type) %>% mutate(folder = paste0(id, "_ATAC"))
data_for_diffbind <- data_for_diffbind %>% mutate(Peaks = paste0(dataset_peaks_location, "/", folder, "/", folder, "_peaks_nosex.narrowPeak"), 
bamReads = paste0(dataset_alignment_location, "/", folder, "/", folder, "_align_filtered_macs2.bam"), PeakCaller = "narrow")

load("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/ATAC_seq_analysis/diffbind_object.Rdata")

# set the parameters right for diffbind as it wants specific names for the variables
data_object$class[DBA_TISSUE, ] = data_for_diffbind$cell_type
data_object$class[DBA_FACTOR, ] = data_for_diffbind$female_sex
data_object$class[DBA_TREATMENT, ] = data_for_diffbind$condition

# using option B 
norm_dba_object <- dba.normalize(data_object, normalize=DBA_NORM_NATIVE,
library=DBA_LIBSIZE_PEAKREADS,background=FALSE)

In [5]:
### Save counts for further analysis
normCounts <- dba.peakset(norm_dba_object, bRetrieve=TRUE, DataType=DBA_DATA_FRAME)
write.csv(normCounts, file="/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/ATAC_seq_analysis/ATAC_DESeq2_normalized_counts.csv")


In [8]:
### also whilst we are here we might as well save the VST counts
res_condition = dba.contrast(norm_dba_object, design = "~ Tissue", contrast = c("Tissue", "CD8", "CD4"))
res_condition = dba.analyze(res_condition)
library("DESeq2")
write.csv(assay(vst(res_condition$DESeq2$DEdata,blind=TRUE)),"/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/ATAC_seq_analysis/ATAC_DESeq2_VST_normalized_counts.csv")


Computing results names...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

